In [1]:
import pandas as pd
import numpy as np
import re

# 读取 Excel 数据
df = pd.read_excel(r'data.xlsx')


# 遍历每一行，找到最后一个非空值并替换为空
for i in range(len(df)):
    # 获取当前行，排除 NaN 值
    row = df.iloc[i].dropna()
    if not row.empty:
        # 找到最后一个非空值的列名
        last_col = row.index[-1]
        # 将该位置的值设为空
        df.at[i, last_col] = None
print(df.head())




# 清理日期列，提取8位数字日期
df['日期'] = df['日期'].str.extract(r'(\d{8})')[0]

# 将日期转换为 datetime 格式并提取月份
df['日期'] = pd.to_datetime(df['日期'], format='%Y%m%d')
df['月份'] = df['日期'].dt.to_period('M')

# 确定留存率列（假设列名形如 '1日', '2日', ...）
retention_cols = [col for col in df.columns if col.endswith('日')]

# 计算每列的非空行数，过滤非空行数 >= 10 的列
na = df[retention_cols].isnull().sum()
valid_cols = [col for col in retention_cols if df[col].notnull().sum() >= 10]
print(f"有效留存率列（非空行数 >= 10）：{valid_cols}")

# 仅保留有效列、日期、激活数和月份
df = df[['日期', '激活数', '月份'] + valid_cols]


# 计算加权留存率
def calculate_weighted_retention(df, group_by=None):
    if group_by:
        grouped = df.groupby(group_by)
    else:
        grouped = [(None, df)]
    
    results = []
    for name, group in grouped:
        weighted_retention = {}
        for col in valid_cols:
            # 只使用非缺失值的行
            valid_rows = group[col].notnull()
            if valid_rows.sum() >= 10:  # 确保分组后非空行数 >= 10
                weighted_retention[col] = (
                    np.sum(group.loc[valid_rows, col] * group.loc[valid_rows, '激活数']) /
                    group.loc[valid_rows, '激活数'].sum() * 100
                )
            else:
                weighted_retention[col] = np.nan  # 分组内非空行数 < 10，设为 NaN
        result = {'分组': name} if group_by else {}
        result.update({col: f"{weighted_retention[col]:.2f}%" if not np.isnan(weighted_retention[col]) else np.nan for col in valid_cols})
        results.append(result)
    
    return pd.DataFrame(results)

# 整体加权留存率
overall_retention = calculate_weighted_retention(df)
print("整体加权留存率：")
print(overall_retention)
overall_retention.to_csv(r'整体.csv')
# 分月加权留存率
monthly_retention = calculate_weighted_retention(df, '月份')
print("\n分月加权留存率：")
print(monthly_retention)
monthly_retention.to_csv(r'分月.csv')

            日期   激活数      1日      2日      3日      4日      5日      6日      7日  \
0  20241101(五)  1109  0.4058  0.3156  0.2831  0.2525  0.2173  0.1921  0.1930   
1  20241102(六)  1008  0.4107  0.3204  0.2718  0.2321  0.2302  0.2153  0.1855   
2  20241103(日)   842  0.4133  0.3052  0.2684  0.2542  0.2399  0.2280  0.2268   
3  20241104(一)   674  0.4065  0.3279  0.2463  0.2226  0.2092  0.2018  0.1973   
4  20241105(二)   737  0.4206  0.3324  0.2754  0.2619  0.2334  0.2076  0.1913   

       8日  ...    171日    172日    173日    174日    175日    176日    177日  \
0  0.1704  ...  0.0207  0.0243  0.0271  0.0325  0.0352  0.0261  0.0252   
1  0.1776  ...  0.0317  0.0347  0.0278  0.0288  0.0248  0.0298  0.0308   
2  0.2078  ...  0.0392  0.0356  0.0356  0.0321  0.0392  0.0368  0.0392   
3  0.1869  ...  0.0237  0.0267  0.0223  0.0178  0.0252  0.0237  0.0193   
4  0.1872  ...  0.0407  0.0312  0.0393  0.0407  0.0366  0.0339  0.0339   

     178日    179日  180日  
0  0.0352  0.0280   NaN  
1  0.0278  0.0268   Na